# Recursion

## Outline

- Motivation

- What is recursion

- Examples of recursion

## Motivation

In any programming language, we want to automate calculations to make our life easier, especially tasks that are repetitive in nature.

In imperative programming languages these repetitive tasks are dealt with using loops, for example in python you can have

```python
for i in [1, 2, 3, 4]:
    print(i)
    
i = 1
while i < 5:
  print(i)
  i += 1
```

Both result in the output of the sequence
```
1
2
3
4
```

Since Haskell is a functional programming language, we do not have these built-in looping functions `for` or `while`.

However, we can't live without cycles in code to automate repetitive tasks, this is where `recursion` helps Haskell out.

<div class="alert alert-block alert-info">
    Fundamentally, imperative and functional language differ in that their core structures are described by two different things.  In a functional language, we only define what things are, in an imperative language we describe how things become.
</div>

## What is recursion precisely

Say we want to calculate the sum of a list of numbers.

Then for an imperative language we set two things, first we start with setting a variable `sum` to zero. Secondly, for each element in the list, we **describe** how to add that element to that sum in incremental steps.

```python
sum = 0
for i in list:
    sum = sum + i

print(sum)
```

In a functional language, we also do two things. First, we **define** what the sum of an empty list is (which is zero). Then we **define** the sum of a non-empty list as the value of the first element plus the sum of the remaining list.

```haskell
sum [] = 0
sum (x:xs) = x + sum xs
```

Visually, this evaluates `[1,2,3,4,5]` as

```haskell
sum [1,2,3,4,5] = 1 + sum [2,3,4,5]
                = 1 + 2 + sum [3,4,5]
                = 1 + 2 + 3 + sum [4,5]
                = 1 + 2 + 3 + 4 + sum [5]
                = 1 + 2 + 3 + 4 + 5 + sum []
                = 1 + 2 + 3 + 4 + 5 + 0
                = 15
```

In the above example we see that recursion is a pattern where a function calls itself! This means that the function name is used somewhere in the function body itself. The simplest example of recursion is the following example

In [ ]:
f x = x + f x
f 1

But notice that when we evaluate this function, it never stops calling itself! The result will be an overflow of computer resources due to the calculation growing too big. This is similar to the more imperative `while true` loop that never stops.

```python
sum = 0
while true:
  sum = sum + 1
```

Though a never-ending loop can be handy, we generally want recursion to stop at some point. This is why we need an edge condition that stops the recursive call of the function, like the `sum [] = 0` in the above example.

To wrap it up, when we are building a recursive function `f` that needs to end at some point, we need to keep two things in mind

   1) What are the set of rules that reduces all successive cases toward the edge case?
   2) What is the edge case to stop the recursion?

## Other recursion examples without numbers

Above we saw examples with lists and numbers, but recursion can cover much more! Let's consider more examples and each time try to answer the above two questions.

**Example 1** Say we want a function `myElem` that checks if an element is in a list. Then we have

  1) An element is in a list if we compare it to the first element `x`, and combine that boolean with the outcome of the element being in the remainder of the list `xs` using `(||)`
  2) The empty list `[]` contains no elements, so it returns `False`

In [ ]:
myElem :: (Eq a) => a -> [a] -> Bool
myElem _ [] = False
myElem a (x:xs) = (a == x) || myElem a xs

myElem 1 [0,3,4,5]
myElem 1 [0,2,1]

Notice that, like in the `sum` example above, we are again traversing the elements of the list from left to right. This is a common pattern in recursion using the pattern matched `x` and `xs` against a list.

**Example 2** Say we want a function `mySeparator` that adds a separator element around all other elements of a list. Then we have

  1) For the first element `x` in the list, we add the separator `a` before that element, then we combine that list with the `mySeparator` acting on the remainder of the list `xs`.
  2) Since we want the list to start and end with the separator, the empty list returns only the separator `a` in a list.

In [ ]:
mySeparator :: a -> [a] -> [a]
mySeparator a [] = [a]
mySeparator a (x:xs) = a : x : mySeparator a xs

mySeparator ' ' "WithSpaces"
mySeparator '-' "WithDashes"

**Example 3** Say we want a function `myTake` that takes the first `n` elements of another list.

  1) Given that `n>0`, we define the initial step on a list as combining the first element `x` with the result of myTake on the rest of the list `xs` with subtracting 1 from `n`.
  2) You cannot take an element of the empty list `[]`, so we define this as the edge case.
  
Additionally, we would like our function to be "well-behaved", that's  why we want to return the empty list if we give is a negative integer.

In [ ]:
myTake :: Int -> [a] -> [a]
myTake n _      | n <= 0 =  []
myTake _ []              =  []
myTake n (x:xs)          =  x : take (n-1) xs        
         
myTake 100 [1,2,3]
myTake 2 [1,2,3]
myTake (-10) [1,2,3]

**Example 4** Say we want a function `myReverse` that reverses a string.

  1) Given a first element `x` we should append that to the result of `myReverse` on the remaining list `xs`.
  2) The reverse of the empty list is itself.

In [ ]:
myReverse :: [a] -> [a]
myReverse [] = []
myReverse (x:xs) = myReverse xs ++ [x]

myReverse "Hello World"
myReverse [1,2,3,4,5]

**Example 5** Say we want a function `myMap` that applies a function `f` to each element of a list.

  1) Given a first element `x` we should apply `f` to that element and prepend that to `myMap` acting on the remainder `xs` with `f`.
  2) You cannot apply `f` on non-existing elements of an empty list, so the empty list returns itself.

In [ ]:
myMap :: (a -> b) -> [a] -> [b]
myMap _ [] = []
myMap f (x:xs) = f x : myMap f xs

myMap (+1) [1,2,3,4]

**Example 6** Say we want a function `orderList` that orders a list of elements from small to big.

  1) Given the first element `x`, we insert it in the ordered version of the remainder `xs` of the list.
  2) The empty list is already ordered.
  
Notice that we need a helper function here, namely a function `myInsert` that can insert an element `n` according to its ordering in a list. This function is also defined recursively!

  1) Given the first element `x`, if that is larger or equal than the element `n`, then `n` should go in front of it. If not, `x` should be prepended with the list that inserts `n` to the remaining list `xs`
  2) we can only insert `n` in the empty list `[]` as `[n]`
  
Together, we get

In [ ]:
myInsert :: (Ord a) => a -> [a] -> [a]
myInsert n [] = [n]
myInsert n (x:xs)
            | n <= x    = n : x : xs
            | otherwise = x : myInsert n xs

orderList :: (Ord a) => [a] -> [a]
orderList [] = []
orderList (x:xs) = myInsert x $ orderList xs

orderList [2,5,1,3,4]

<div class="alert alert-block alert-info">
    The compiler suggests a better way to write our <code>orderList</code> function using <code>foldr</code>. This is a pattern that we will see in the next lesson, it is handy when a function is "folded" over another type. Here, folded means that we can view this recursive structure as first sorting the first element with the empty list. Then insert the second element to that just gotten sorted list, and so on.
</div>

**Example 7** Remember that in Haskell everything is a function! Even our objects are functions that can define themselves using recursion. Consider for example the even numbers that can be recursively defined as base value 0 with the additional step of using `myMap` to the add 2 to itself. This will generate an infinite list of the even numbers and never stop!

In [ ]:
numbers1 :: [Int]
numbers1 = 0 : myMap (+2) numbers1
myTake 10 numbers1

**Example 8** Say we want the list of Fibonacci numbers. These are an infinite sequence of numbers that follow the pattern that in any segment of the sequence we have that `a : b : a+b` and have a base value of 0 and 1. We can convert this to an infinite list with

In [ ]:
fib a b = a : fib b (a+b)
myTake 10 $ fib 0 1